In [1]:
# # Webcam Photo Capture (Fixed Live Preview)
# 
# This notebook allows you to:
# - Display live webcam feed
# - Capture photos
# - Save images with custom names
# - View captured photos

# %%
# Section 1: Setup and Imports
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from datetime import datetime
import os
import threading

# Create output directory if it doesn't exist
os.makedirs('captured_photos', exist_ok=True)

# Global variable to control preview thread
preview_active = False



In [2]:
# %%
# Section 2: Camera Initialization
def init_camera():
    camera = cv2.VideoCapture(0, cv2.CAP_V4L2)
    if not camera.isOpened():
        raise Exception("Could not open camera")
    return camera

camera = init_camera()


In [3]:
# %%
# Section 3: Utility Functions
def bgr8_to_jpeg(frame, quality=75):
    """Convert OpenCV BGR8 format to JPEG bytes"""
    return bytes(cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])[1])

def show_frame(frame):
    """Display frame in the notebook"""
    rotated_frame = rotate_frame(frame, angle=0)  # Rotate 90 degrees
    image_widget.value = bgr8_to_jpeg(rotated_frame)

def rotate_frame(frame, angle=90):
    """Rotate the frame by specified angle (90, 180, or 270 degrees)"""
    if angle == 90:
        return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(frame, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return frame

In [4]:
# %%
# Section 4: Live Preview (Fixed)
def live_preview():
    """Show live camera feed"""
    global preview_active
    preview_active = True
    try:
        while preview_active:
            ret, frame = camera.read()
            if not ret:
                print("Failed to grab frame")
                break
            show_frame(frame)
            time.sleep(0.033)  # ~30 FPS
            
    except Exception as e:
        print(f"Preview error: {str(e)}")
    finally:
        print("Preview stopped")

# Create and display control buttons
start_button = widgets.Button(description="Start Preview")
stop_button = widgets.Button(description="Stop Preview")
display(widgets.HBox([start_button, stop_button]))

def on_start_click(b):
    global preview_active
    if not preview_active:
        threading.Thread(target=live_preview, daemon=True).start()

def on_stop_click(b):
    global preview_active
    preview_active = False

start_button.on_click(on_start_click)
stop_button.on_click(on_stop_click)

display(widgets.HTML("<hr>"))  # Visual separator


HTML(value='<hr>')

In [5]:
# %%
# Section 5: Photo Capture
filename_input = widgets.Text(
    value='photo',
    placeholder='Enter filename (without extension)',
    description='Filename:',
    disabled=False
)

capture_button = widgets.Button(description="Capture Photo")
display(widgets.VBox([filename_input, capture_button]))

# Store captured images
captured_images = []

capture_output = widgets.Output()
display(capture_output)

def on_capture_click(b):
    with capture_output:
        clear_output(wait=True)
        ret, frame = camera.read()
        if ret:
            frame = rotate_frame(frame, angle=0)  # Keep your rotation if needed
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{filename_input.value}_{timestamp}.jpg"
            filepath = os.path.join('captured_photos', filename)
            
            cv2.imwrite(filepath, frame)
            captured_images.append(filepath)
            
            print(f"Photo saved as: {filepath}")
            #display(widgets.Image(value=bgr8_to_jpeg(frame), width=300))
        else:
            print("Failed to capture photo")

capture_button.on_click(on_capture_click)

# Image display widget
image_widget = widgets.Image(format='jpeg', width=600, height=500)
display(image_widget)

Output()

Image(value=b'', format='jpeg', height='500', width='600')

Preview stopped


In [6]:
# %%
# Section 6: View Captured Photos
if captured_images:
    print("Recently captured photos:")
    for img_path in captured_images[-3:]:  # Show last 3 photos
        display(widgets.Image(value=open(img_path, 'rb').read(), width=300))
        print(img_path)
else:
    print("No photos captured yet")



Recently captured photos:


FileNotFoundError: [Errno 2] No such file or directory: 'captured_photos/photo_20250802_214415.jpg'

In [6]:
# %%
# Section 7: Cleanup
def cleanup():
    global preview_active
    preview_active = False
    camera.release()
    print("Camera released")

cleanup()

Camera released
